# Assignment 3 
u6664231

## Question 1  Git in practice 

See commit history on public repo: https://github.com/gkb21/ASTR4004_assignment3

Also note for the first dozen or so commits I was playing around with how branching and merging worked and what I could and couldn't do so sorry for the chaos  

## Question 2 Using ADQL to Search for Bright Stars Around the Open Cluster M67

A colleague is interested in the open cluster Messier 67 (RA = 132.825 deg, Dec = 11.8 deg)
and is considering an observation proposal using the 2dF fibre positioner and HERMES
spectrograph (effective for Gaia G band magnitudes < 14). They need to know if there
are enough bright stars in this region for observation. Your task is to assist by querying
data from Gaia DR3 and performing some basic analysis.

- #### Download all stars within 1 degree of the center of Messier 67 that are brighter than G = 14 in Gaia DR3 (the table is called gaiadr gaia source) and include a crossmatch these stars with the 2MASS catalog and report your ADQL query text (use the """query""" notation)

In [48]:
from astroquery.gaia import Gaia
import pandas as pd
from astropy.table import Table

# Define the center of Messier 67
ra_m67 = 132.825   # Right Ascension in degrees
dec_m67 = 11.8 # Declination in degrees
radius = 1.0  # Search radius in degrees
mag_limit = 14  # Magnitude limit

'''
This is a query for stars that are 1 degree of the center of Messier 67 that are brighterthan G = 14
from Gaia DR3 with a 2mass crossmatch
'''
gaia_2mass_crossmatch_query = f"""
SELECT *
FROM gaiadr3.gaia_source AS gaia
JOIN gaiaedr3.tmass_psc_xsc_best_neighbour AS xmatch USING (source_id)
JOIN gaiaedr3.tmass_psc_xsc_join AS xjoin USING (clean_tmass_psc_xsc_oid)
JOIN gaiadr1.tmass_original_valid AS tmass
    ON xjoin.original_psc_source_id = tmass.designation

WHERE CONTAINS(
        POINT(gaia.ra, gaia.dec), 
        CIRCLE({ra_m67}, {dec_m67}, {radius})
    ) = 1
    AND gaia.phot_g_mean_mag < {mag_limit}
ORDER BY gaia.phot_g_mean_mag

"""

# Runs the ADQL query for 'gaia_2mass_crossmatch_query'
job = Gaia.launch_job(gaia_2mass_crossmatch_query ) 
results = job.get_results()
# Convert to pandas data frame   
gaia_2mass_crossmatch_query_data_table = results.to_pandas()
print(gaia_2mass_crossmatch_query_data_table)


2119545319.py:               solution_id                  DESIGNATION  random_index  \
0     1636148068921376768  Gaia DR3 608020176290124544     365257723   
1     1636148068921376768  Gaia DR3 598677041873269888     347884130   
2     1636148068921376768  Gaia DR3 604920240695111424     441504422   
3     1636148068921376768  Gaia DR3 604992258706635520     554171204   
4     1636148068921376768  Gaia DR3 604684326730942592     803047084   
...                   ...                          ...           ...   
1013  1636148068921376768  Gaia DR3 608127481752974720     135826423   
1014  1636148068921376768  Gaia DR3 604901652076632960    1318611312   
1015  1636148068921376768  Gaia DR3 604972226979105664     837264516   
1016  1636148068921376768  Gaia DR3 607987736401416576     711162152   
1017  1636148068921376768  Gaia DR3 598890381488889088     503374393   

      ref_epoch          ra  ra_error        dec  dec_error   parallax  \
0        2016.0  132.164662  0.035635  12.5469

- #### Determine how many stars are returned from the initial query.a

In [29]:
# Determines how many stars are in the filtered data set via checking the len of the amount of entries
num_stars = len(gaia_2mass_crossmatch_query_data_table)
print(f"Number of stars brighter than G={mag_limit} within 1 degree of M67: {num_stars}") # prints number of stars 

1491207227.py: Number of stars brighter than G=14 within 1 degree of M67: 1018


- #### Identify any stars with bad 2MASS photometry, where ph qual is not ’AAA’.

In [46]:
'''
Filter the stars where ph_qual is NOT 'AAA' leaving behind 
only the bad 2MASS photometry stars using pandas query
'''
bad_photometry_stars = gaia_2mass_crossmatch_query_data_table.query("ph_qual != 'AAA'")
# Determines how many stars are in the filtered data set via checking the len of the amount of entries
num_bad_photometry_stars = len(bad_photometry_stars)
print(num_bad_photometry_stars) # prints the number of bad 2MASS photometry stars
print(bad_photometry_stars) # prints a table of bad 2MASS photometry stars 


3193909364.py: 21
3193909364.py:              solution_id                  DESIGNATION  random_index  \
0    1636148068921376768  Gaia DR3 608020176290124544     365257723   
1    1636148068921376768  Gaia DR3 598677041873269888     347884130   
4    1636148068921376768  Gaia DR3 604684326730942592     803047084   
22   1636148068921376768  Gaia DR3 605101037342309376    1275034334   
24   1636148068921376768  Gaia DR3 598689102141448320     446746802   
25   1636148068921376768  Gaia DR3 598955115237068032     594867401   
69   1636148068921376768  Gaia DR3 604950271106216064     156518440   
97   1636148068921376768  Gaia DR3 604911268507711232    1004522431   
173  1636148068921376768  Gaia DR3 604917835513458688     941052385   
314  1636148068921376768  Gaia DR3 604713459494948352     532598536   
403  1636148068921376768  Gaia DR3 598689102142789760    1573797226   
435  1636148068921376768  Gaia DR3 604636321880758144     383307719   
464  1636148068921376768  Gaia DR3 598921030

In [57]:
# All the stars with bad 2MASS photometry
print(bad_photometry_stars['DESIGNATION'])

56804046.py: 0      Gaia DR3 608020176290124544
1      Gaia DR3 598677041873269888
4      Gaia DR3 604684326730942592
22     Gaia DR3 605101037342309376
24     Gaia DR3 598689102141448320
25     Gaia DR3 598955115237068032
69     Gaia DR3 604950271106216064
97     Gaia DR3 604911268507711232
173    Gaia DR3 604917835513458688
314    Gaia DR3 604713459494948352
403    Gaia DR3 598689102142789760
435    Gaia DR3 604636321880758144
464    Gaia DR3 598921030375475200
558    Gaia DR3 604917831219174528
569    Gaia DR3 604921202767811712
592    Gaia DR3 598925943818062208
641    Gaia DR3 604904847532342528
660    Gaia DR3 598896841119762816
803    Gaia DR3 604904847532342272
900    Gaia DR3 598921030375475072
928    Gaia DR3 608019317296666624
Name: DESIGNATION, dtype: object


- #### Identify any stars with negative (or non-positive) parallaxes in the Gaia data.

In [47]:
# Filter to leave only negative parallaxes and null parallaxes in our data set using pandas query 
negative_parallax_stars = gaia_2mass_crossmatch_query_data_table.query('parallax.isnull() or parallax <= 0')
# Determines how many stars are in the filtered data set via checking the len of the amount of entries 
num_bad_photometry_stars = len(negative_parallax_stars)
print(num_bad_photometry_stars) #prints the num of stars with egative parallaxes and null parallaxes
print(negative_parallax_stars) # prints pandas table of stars 

3575783184.py: 9
3575783184.py:              solution_id                  DESIGNATION  random_index  \
86   1636148068921376768  Gaia DR3 604917663716360576     324647624   
155  1636148068921376768  Gaia DR3 604921546365012480     131241535   
179  1636148068921376768  Gaia DR3 604921542069455104     544905365   
243  1636148068921376768  Gaia DR3 598886739356653696    1501496757   
295  1636148068921376768  Gaia DR3 598873953238690816      49906729   
534  1636148068921376768  Gaia DR3 604918282190043648    1391376225   
754  1636148068921376768  Gaia DR3 598847569256078464     314066461   
832  1636148068921376768  Gaia DR3 598892580511613312     683012459   
945  1636148068921376768  Gaia DR3 598892580512198144    1004824233   

     ref_epoch          ra   ra_error        dec  dec_error  parallax  \
86      2016.0  132.861743   7.256874  11.811254   3.914955       NaN   
155     2016.0  132.836633   5.527748  11.890591   3.062133       NaN   
179     2016.0  132.836514   0.798105 

- #### Apply these two quality cuts (removing stars with bad 2MASS photometry and non-positive parallaxes). After applying the cuts, determine how many stars remain.

In [63]:
'''
Filter out both negative parallaxes and null parallaxes using pandas query as well
as making sure that stars don't have bad 2MASS photometry using pandas query 
'''
filtered_gaia_2mass_crossmatch_query_data= gaia_2mass_crossmatch_query_data_table.query("ph_qual == 'AAA' and parallax.notnull() and parallax > 0")
# Determines how many stars are in the filtered data set via checking the len of the amount of entries
filtered_num_stars = len(filtered_gaia_2mass_crossmatch_query_data)
# prints the amount of filtered stars 
print(f"Number of stars brighter than G={mag_limit} within 1 degree of M67 that are ph_qual AAA quality and parallax is not <0 or nan: {filtered_num_stars}")


3638111359.py: Number of stars brighter than G=14 within 1 degree of M67 that are ph_qual AAA quality and parallax is not <0 or nan: 988


- #### Using the remaining stars, generate a figure with two panels (1 point per panel):


(a) A color-magnitude diagram (CMD) of Gaia BP-RP vs. absolute G magnitude.


(b) A 2MASS J-Ks vs. apparent K magnitude diagram.

- #### Save the figure as figures/cmds M67.png with a resolution of 200 dots per inch

- #### Give your colleague a recommendation for the potential proposal when only judging the fibre usage.

## Question 3 The radial metallicity relation in simulated data 

The radial metallicity relation is a function that describes the change of metallicity -
here the gas phase metallicity $A(O) = log10(NO/NH) + 12$ - along the galactocentric
radius $R_{Gal}$.. Understanding the radial metallicity gradient in galaxies provides critical
insights into their formation and evolutionary processes, such as inside-out formation,
gas accretion, outflows, and radial migration. A lot of work has been done through
observational studies (e.g. Ho et al., 2017, ApJ, 846, 39) and a few simulations (e.g.
Grand et al., 2016, MNRAS, 460, 94), but more works needs to be done to understand
the radial metallicity gradient!
Your colleague has just finished an exciting cosmological simulation that traces the
gas phase metallicity for a Milky Way analogue, that is, a spiral galaxy. They have
limited the simulation data to the positions (x, y, z) of the innermost gas particles
(RGal. < 25 kpc) and their gas phase metallicity A(O) and uploaded them as a FITS
file for you here: https://github.com/svenbuder/astr4004_2024_week7/blob/main/data/nihao_uhd_simulation_g8.26e11_xyz_positions_and_oxygen_ao.fits

In [9]:
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Make the size and fonts larger for this presentation
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2

In [10]:
import pandas as pd
from astropy.table import Table # this works well for FITS data catalogues
from astropy.io import fits # this is your more agnostic way to work for FITS images
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np

#### - Download the file from the link above into **data/**. Load the file with python and then perform the following tasks to create figures that are saved in **figures/**:

In [ ]:
# FITS Images
fits_file = fits.open('data/nihao_uhd_simulation_g8.26e11_xyz_positions_and_oxygen_ao.fits', ignore_missing_simple=True)
print(fits_file)

In [ ]:
#we can also just dump information for this fits file
print(fits_file.info())

In [ ]:
#print the header for the Primary Extension
print(fits_file[0].header.items)

In [ ]:
data = fits_file[1].data
data_table = Table(data)
print(data_table.colnames)

(a) Logarithmic density plot of RGal. vs. A(O), with a linear fit and legend.

In [15]:
# Calculate RGal from x, y, z positions
x = data_table['x']
y = data_table['y']
z = data_table['z']
A_O = data_table['A_O']  # Adjust this based on actual column name
# Calculate RGal
RGal = np.sqrt(x**2 + y**2 + z**2)

In [16]:
# Filter for RGal < 25 kpc
mask = RGal < 25
RGal_filtered = RGal
A_O_filtered = A_O

In [17]:
# Prepare the linear regression model
model = LinearRegression()
model.fit(RGal_filtered.reshape(-1, 1), A_O_filtered)
# Predict metallicity for the fitted model
A_O_fit = model.predict(RGal_filtered.reshape(-1, 1))

In [18]:
# Linear fit
linear_fit = np.polyfit(RGal, A_O, 1)  # 1 for linear
m, b = linear_fit  # slope and intercept

In [19]:
import statsmodels.api as sm

# Linear model
x_with_intercept = sm.add_constant(RGal)  # Adds the intercept term
linear_model = sm.OLS(A_O, x_with_intercept).fit()

In [ ]:
# Print linear model summary (with statistical info)
print(linear_model.summary())
x_smooth = np.linspace(min(RGal), max(RGal), 500)
linear_fit = linear_model.params[0] + linear_model.params[1] * x_smooth


In [ ]:
residuals = A_O - linear_model.predict(x_with_intercept)
plt.scatter(RGal, residuals, s=2, alpha=1)
plt.axhline(0, color='red', linestyle='--', label='Zero Residual')

In [ ]:

plt.hexbin(RGal, residuals, gridsize=100, cmap='inferno')
plt.colorbar(label='Counts')
plt.axhline(0, color='red', linestyle='--', label='Zero Residual')

In [23]:
from scipy.optimize import curve_fit

# Define linear model
def linear_model(x, m, b):
    return m * x + b

In [24]:
popt_linear, pcov_linear = curve_fit(linear_model, RGal, A_O, absolute_sigma=True)
m_linear, b_linear = popt_linear
perr_linear = np.sqrt(np.diag(pcov_linear))

In [ ]:
print(f"Linear Fit: y = {m_linear:.2f}x + {b_linear:.2f}")
print(f"Linear Fit uncertainties: m = {perr_linear[0]:.2f}, b = {perr_linear[1]:.2f}")

In [ ]:
# Create the plot
# Create the figure
fig = plt.figure(figsize=(15, 6))
ax1 = fig.add_subplot(121)  # First subplot
ax2 = fig.add_subplot(122)  # Second subplot


hb = ax1.hexbin(RGal_filtered, A_O_filtered, gridsize=70, cmap='inferno', bins='log')
ax1.plot(RGal_filtered, A_O_fit, color=(0.078, 0.667, 1.0), linewidth=5,label='Linear Fit')
ax1.plot(x_smooth, linear_fit, label='Linear Fit', color='red')
ax1.set_xlabel('R_Gal (kpc)')
ax1.set_ylabel('A(O)')
ax1.set_title('Logarithmic Density Plot of RGal vs. A(O)')

ax2.scatter(RGal, residuals, s=2, alpha=1)
ax2.axhline(0, color='red', linestyle='--', label='Zero Residual')
ax2.set_xlabel('RGal (kpc)')
ax2.set_ylabel('Residuals')
ax2.set_title('Residuals Plot')




#fit_info = f'Slope: {m:.2f}, Intercept: {b:.2f}, R²: {r_value**2:.2f}'
#ax1.legend([fitline[0]], [fit_info], loc='lower left')
plt.colorbar(hb, ax=ax1, label='Log Density')
plt.savefig('figures/RGal_vs_AO.png', bbox_inches='tight')
plt.show()
# Save the figure

plt.close()  # Close the figure to free memory

In [ ]:
# Create the plot
# Create the figure
fig = plt.figure(figsize=(15, 6))
ax1 = fig.add_subplot(121)  # First subplot
ax2 = fig.add_subplot(122)  # Second subplot


hb = ax1.hexbin(RGal_filtered, A_O_filtered, gridsize=70, cmap='inferno', bins='log')
ax1.plot(RGal_filtered, A_O_fit, color=(0.078, 0.667, 1.0), linewidth=5,label='Linear Fit')
ax1.plot(x_smooth, linear_fit, label='Linear Fit', color='red')
ax1.set_xlabel('R_Gal (kpc)')
ax1.set_ylabel('A(O)')
ax1.set_title('Logarithmic Density Plot of RGal vs. A(O)')

hb2 = ax2.hexbin(RGal, residuals, gridsize=100, cmap='inferno')
ax2.axhline(0, color='red', linestyle='--', label='Zero Residual')
ax2.set_xlabel('RGal (kpc)')
ax2.set_ylabel('Residuals')
ax2.set_title('Residuals Plot')


#fit_info = f'Slope: {m:.2f}, Intercept: {b:.2f}, R²: {r_value**2:.2f}'
#ax1.legend([fitline[0]], [fit_info], loc='lower left')
plt.colorbar(hb, ax=ax1, label='Log Density')
plt.colorbar(hb2, ax=ax2, label='Counts')
plt.savefig('figures/RGal_vs_AO.png', bbox_inches='tight')
plt.show()
# Save the figure

plt.close()  # Close the figure to free memory

(b) Residuals of the fit, $R_{Gal}$. vs. $\Delta A(O)$.

#### Use a python fitting tool to fit a linear function to the data, reporting the intercept and slope with uncertainties. Include any hyperparameters used.

#### Discuss where the linear model fits well and where it does not. Use statistical metrics, such as the root mean squares or other goodness-of-fit indicators, to quantify the performance of your linear fit in general and regions with larger residuals.

#### Plot a 3-panel figure for the x vs. y plane using the same bins and sensible colormaps:

(a) 2D-histogram of the median simulated A(O)

(b) 2D-histogram of the median fitted A(O)

(c) 2D-histogram of the median residuals ∆A(O)

#### Describe your choice of 2D bins. Discuss what details would be missed with fewer bins or problems encountered with more bins.

#### Analyze the residuals in more detail and propose an explanation for any patterns you observe.

In [28]:
fits_file.close()